In [28]:
from copy import deepcopy
from pathlib import Path

import bw2data
import pandas as pd

from enbios import Experiment, report
from enbios.base.models import ExperimentData

In this demo, we demonstrate the basic working of Enbios. There are 4 main parts to this introduction. The first three parts, incrementally extend the experiment, while the 4. part displays some useful functionalities, that helps to understand how to configure enbios.

For this introduction, we are going to use Brightway to do LCA calculations of ecoinvent (v.3.9.1) activities, that are at the bottom of the MuSIASEM hierarchy, so one should have a brigthway project, with some the ecoinvent dataset created before starting.

In the first step, we get some brighway activities are construct the enbios configuration step by step.

In [29]:
# get an overview of brightway projects and databases
report()

In [30]:
# select the brightway project and database (e.g. some ecoinvent database)
PROJECT_NAME = "ecoinvent_391"
DATABASE = "ecoinvent_391_cutoff"

bw2data.projects.set_current(PROJECT_NAME)
db = bw2data.Database(DATABASE)

# Simple example experiment
Let's get 2 wind turbines from Spain

In [31]:
wind_turbines_spain = db.search(
    "electricity production, wind, 1-3MW turbine, onshore", filter={"location": "ES"}
)[:2]
wind_turbines_spain

In [32]:
# Now we use those, to define 2 leaf-nodes in our hierarchy.
experiment_nodes = []

for activity in wind_turbines_spain:
    experiment_nodes.append(
        {
            "name": activity["name"],
            "adapter": "brightway-adapter",
            "config": {"code": activity["code"]},
        }
    )

In [33]:
# we can modify the output of the activities, by default it is the reference product (1 of the activity unit)
experiment_nodes[0]["config"]["default_output"] = {
    "unit": "kilowatt_hour",
    "magnitude": 3,
}
experiment_nodes

In [34]:
hierarchy = {
    "name": "root",
    "aggregator": "sum-aggregator",
    "children": experiment_nodes,
}

In [35]:
# select 2 random methods and convert them into the form for enbios2
experiment_methods = [bw2data.methods.random() for _ in range(2)]

experiment_methods = {m[-1]: m for m in experiment_methods}
experiment_methods

In [36]:
# todo, demo on how to select certain methods
bw2data.methods

In [37]:
# alternatively, we could just specify two methods
experiment_methods = {
    "GWP1000": (
        "ReCiPe 2016 v1.03, midpoint (H)",
        "climate change",
        "global warming potential (GWP1000)",
    ),
    "LOP": (
        "ReCiPe 2016 v1.03, midpoint (E)",
        "land use",
        "agricultural land occupation (LOP)",
    ),
    "WCP": (
        "ReCiPe 2016 v1.03, midpoint (E)",
        "water use",
        "water consumption potential (WCP)",
    ),
}

In [38]:
# let's store the raw data, because we want to modify it later
simple_raw_data = {
    "adapters": [
        {
            "adapter_name": "brightway-adapter",
            "config": {"bw_project": PROJECT_NAME},
            "methods": experiment_methods,
        }
    ],
    "hierarchy": hierarchy,
}

# make a first validation of the experiment data
exp_data = ExperimentData(**simple_raw_data)
exp_data.model_dump(exclude_unset=True)

In [39]:
# create experiment object. This will validate the activities, their outputs, the methods and the scenarios.
simple_experiment: Experiment = Experiment(simple_raw_data)

In [40]:
print(simple_experiment.info())

## Running the experiment

In [41]:
# run all scenarios at once, Results will be returned as dictionary
# <scenario_name : str> : <scenario_result : dict>
# <scenario_result> : dict represents the root node of the results tree, with the following keys:
# name, output, results, children
# where results, is a dictionary of impacts to dict: unit, magnitude | multi_magnitude
# The first and only positional parameter 'results_as_dict' can be set to False, to get the tree unserialized as :  BasicTreeNode[ScenarioResultNodeData]
results = simple_experiment.run()

# Result
The result is a dictionary of scenario names, where for each scenario we have a tree (representing the activity hierarchy). Each node (`BasicTreeNode`) in the tree has a `data` object, which is of the type `ScenarioResultNodeData`, which have the fields `output`, `result`.

In [42]:
results

In [43]:
# we can dump the results into a csv file
simple_experiment.results_to_csv("test.csv")
pd.read_csv("test.csv").fillna("")

In [44]:
simple_experiment.scenarios[0].result_to_dict()

## Add a technology hierarchy (dendrogram) 
Let's now add a few more activities to the experiment and create a hierarchy of activities.

In [45]:
solar_spain = db.search("solar", filter={"location": "ES"})[:2]
solar_spain

In [46]:
experiment_nodes_solar = []
for activity in solar_spain:
    experiment_nodes_solar.append(
        {"name": activity["name"], "adapter": "bw", "config": {"code": activity["code"]}}
    )

hierarchy_raw_data = deepcopy(simple_raw_data)

hierarchy_raw_data["hierarchy"] = {
    "name": "root",
    "aggregator": "sum",
    "children": [
        {"name": "wind", "aggregator": "sum", "children": experiment_nodes},
        {"name": "solar", "aggregator": "sum", "children": experiment_nodes_solar},
    ],
}

In [47]:
hierarchy_raw_data

In [48]:
hierarchy_experiment: Experiment = Experiment(hierarchy_raw_data)

# Run the 2nd experiment

In [49]:
hierarchy_experiment.run()

In [50]:
# optional storing, to use it in other notebooks
import pickle

pickle.dump(hierarchy_experiment, open("exp.pickle", "bw"))

In [51]:
# print(json.dumps((exp.scenarios[0].result_to_dict()), indent=2))
hierarchy_experiment.scenarios[0].result_to_csv(
    "test.csv", level_names=["root", "technology", "activity"]
)
pd.read_csv("test.csv").fillna("")

## Create several scenarios

In [52]:
from typing import Optional
from random import randint


def create_normal_scenario():
    return {
        "name": "normal scenario",
        "nodes": {
            act: ("kilowatt_hour", 1)
            for act in hierarchy_experiment.structural_nodes_names
        },
    }


def create_random_scenario(scneario_name: Optional[str] = None):
    return {
        "name": scneario_name,
        "nodes": {
            act: ("kilowatt_hour", randint(1, 10))
            for act in hierarchy_experiment.structural_nodes_names
        },
    }


scenarios_raw_data = deepcopy(hierarchy_raw_data)
scenarios_raw_data["scenarios"] = [
    create_normal_scenario(),
    create_random_scenario(),
    create_random_scenario(),
]

scenarios_raw_data["scenarios"][0]

In [53]:
scenarios_raw_data

In [54]:
scenarios_experiment = Experiment(scenarios_raw_data)

 ## Run the experiment for the 3rd time
This time will likely take some more time since we need to run 2 scenarios. 

In [55]:
_ = scenarios_experiment.run()
# don't print the whole result

In [56]:
scenarios_experiment.scenarios[0].results_to_csv(
    "s1.csv", level_names=["root", "technology", "activity"]
)
pd.read_csv("s1.csv").fillna("")
Path("s1.csv").unlink()  # delete the file again

We can also now run any new scenario configuration for the given experiment using `run_scenario_config`

In [ ]:
new_random_scenario = create_random_scenario()
new_random_scenario["name"] = "new scenario"
new_results = scenarios_experiment.run_scenario_config(new_random_scenario)

## Inspecting the results

We can now do some transformations of the results. For that is useful to know how to retrieve is singular result from a scenario result. 
The result of a scenario is a tree structure, where the nodes `name`s are activity aliases or names defined in the hierarchy. With the function of BasicTreeNode.find_child_by_name we can directly access the result of a node.    

Following we transform the results into a dictionary of the following structure:
```json
{
    "node": {
        "method": "[list of results for each scenario]"
    }
}
```

In [ ]:
all_results = {}
for node in scenarios_experiment.structural_nodes_names:
    all_results[node] = {method: [] for method in scenarios_experiment.method_names}
    for scenario in scenarios_experiment.scenarios:
        activity_result = scenario.result_tree.find_subnode_by_name(node)
        for method, score in activity_result.data.results.items():
            all_results[node][method].append(score.model_dump(exclude_defaults=True))

all_results